In [1]:
# setup library imports
import io, time, json
import requests
from pathlib import Path
from bs4 import BeautifulSoup
from testing.testing import test

## Handle Basic HTTP Requests


In [2]:

def retrieve_html(url):
    """
    Return the raw HTML at the specified URL.

    Args:
        url (string): 

    Returns:
        status_code (integer):
        raw_html (string): the raw HTML content of the response, properly encoded according to the HTTP headers.
    """
    
    response = requests.get(url)
    return response.status_code, response.text


## Read yelp API key

In [3]:
def read_api_key(filepath="yelp_api_key.txt"):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    # Feel free to modify this function if you are storing the API Key differently
    return Path(filepath).read_text().strip()

In [7]:
## Get yelp business Info

In [8]:
def yelp_search_test(yelp_search):
    total, business = yelp_search(read_api_key(), "Pittsburgh")
    test.true(abs(total - 2600) < 60)
    expected_keys = ['id', 'name', 'phone', 'review_count']
    if len(business):
        test.true(all(k in set(business[0].keys()) for k in expected_keys))

@test
def yelp_search(api_key, query):
    """
    Make an authenticated request to the Yelp API.

    Args:
        query (string): Search term

    Returns:
        total (integer): total number of businesses on Yelp corresponding to the query
        businesses (list): list of dicts representing each business
    """
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization" : "Bearer %s" % (api_key)}
    params = {"location" : query}
    response = requests.get(url, params = params, headers = headers)
    result = json.loads(response.text)

    return result["total"], result["businesses"]

### TESTING yelp_search: PASSED 2/2
###



In [5]:
@test
def all_restaurants(api_key, query):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of dicts representing each business
    """
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization" : "Bearer %s" % (api_key)}
    offset = 0
    params = {"location" : query, "categories": "restaurants", "limit": 40}
    response = requests.get(url, params = params, headers = headers)
    result = json.loads(response.text)
    total = result["total"]
    final = []
    
    while offset<total:
        params["offset"] = offset
        response = requests.get(url, params = params, headers = headers)
        offset += 40
        result = json.loads(response.text)
        final += result["businesses"]
        time.sleep(0.2)
    return final

def all_restaurants_test(all_restaurants):
    res = ['Morcilla', 'Umami', 'Piccolo Forno']
    data = all_restaurants(read_api_key(), 'Polish Hill, Pittsburgh')
    names = [x['name'] for x in data]
    
    for ele in res:
        test.true(ele in names)



### TESTING all_restaurants: PASSED 0/0
# Cannot locate test function named all_restaurants_test
###



---

## Q 2.5: Parse the API Responses and Extract the URLs

Because we want to seperate the __downloading__ from the __parsing__, fill in the following function to parse the URLs pointing to the restaurants on `yelp.com`. As input your function should expect a string of [properly formatted JSON](http://www.json.org/) (which is similar to __BUT__ not the same as a Python dictionary) and as output should return a Python list of strings. The input JSON will be structured as follows (same as the [sample](https://www.yelp.com/developers/documentation/v3/business_search) on the Yelp API page):

In [6]:
json_src = """{
  "total": 8228,
  "businesses": [
    {
      "rating": 4,
      "price": "$",
      "phone": "+14152520800",
      "id": "four-barrel-coffee-san-francisco",
      "is_closed": false,
      "categories": [
        {
          "alias": "coffee",
          "title": "Coffee & Tea"
        }
      ],
      "review_count": 1738,
      "name": "Four Barrel Coffee",
      "url": "https://www.yelp.com/biz/four-barrel-coffee-san-francisco",
      "coordinates": {
        "latitude": 37.7670169511878,
        "longitude": -122.42184275
      },
      "image_url": "http://s3-media2.fl.yelpcdn.com/bphoto/MmgtASP3l_t4tPCL1iAsCg/o.jpg",
      "location": {
        "city": "San Francisco",
        "country": "US",
        "address2": "",
        "address3": "",
        "state": "CA",
        "address1": "375 Valencia St",
        "zip_code": "94103"
      },
      "distance": 1604.23,
      "transactions": ["pickup", "delivery"]
    }
  ],
  "region": {
    "center": {
      "latitude": 37.767413217936834,
      "longitude": -122.42820739746094
    }
  }
}
"""

In [7]:
def parse_api_response_test(parse_api_response):
    test.equal(parse_api_response(json_src), ['https://www.yelp.com/biz/four-barrel-coffee-san-francisco'])

@test
def parse_api_response(data):
    """
    Parse Yelp API results to extract restaurant URLs.
    
    Args:
        data (string): String of properly formatted JSON.

    Returns:
        (list): list of URLs as strings from the input JSON.
    """
    
    return [ele["url"] for ele in json.loads(data)["businesses"]]

### TESTING parse_api_response: PASSED 1/1
###



---

As we can see, JSON is quite trivial to parse (which is not the case with HTML as we will see in a second) and work with programmatically. This is why it is one of the most ubiquitous data serialization formats (especially for RESTful APIs) and a huge benefit of working with a well defined API if one exists. But APIs do not always exists or provide the data we might need, and as a last resort we can always scrape web pages...

## Working with Web Pages (and HTML)

Think of APIs as similar to accessing a application's database itself (something you can interactively query and receive structured data back). But the results are usually in a somewhat raw form with no formatting or visual representation (like the results from a database query). This is a benefit _AND_ a drawback depending on the end use case. For data science and _programatic_ analysis this raw form is quite ideal, but for an end user requesting information from a _graphical interface_ (like a web browser) this is very far from ideal since it takes some cognitive overhead to interpret the raw information. And vice versa, if we have HTML it is quite easy for a human to visually interpret it, but to try to perform some type of programmatic analysis we first need to parse the HTML into a more structured form.

As a general rule of thumb, if the data you need can be accessed or retrieved in a structured form (either from a bulk download or API) prefer that first. But if the data you want (and need) is not as in our case we need to resort to alternative (messier) means.

Going back to the "hello world" example of question 1 with the NYT, we will do something similar to retrieve the HTML of the Yelp site itself (rather than going through the API) programmatically as text. 

---

## Q3: Parse a Yelp restaurant Page

Using `BeautifulSoup`, parse the HTML of a single Yelp restaurant page to extract the reviews in a structured form as well as the total number of pages. Fill in following function stubs to parse a single page of reviews and return:
* the reviews as a structured Python dictionary
* the total number of pages of reviews.

For each review be sure to structure your Python dictionary as follows (to be graded correctly). The order of the keys doesn't matter, only the keys and the data type of the values:

```python
{
    'author': 'Aaron W.' # str
    'rating': 4.0        # float
    'date': '2019-01-03' # str, yyyy-mm-dd
    'description': "Wonderful!" # str
}
```

Return reviews in the order that they are present on the page.

There can be issues with Beautiful Soup using various parsers, for maximum conpatibility (and fewest errors) initialize the library with the default (and Python standard library parser): `BeautifulSoup(markup, "html.parser")`. You may notice that the HTML is automatically generated. Yelp uses a modern web application technology called [React](https://reactjs.org/), which generates the markup from Javascript code. This is a common hazard of scraping data from HTML.

Here's a hint for this problem: [`aria`](https://www.w3.org/WAI/PF/aria-1.1/terms) is a web standard that supports text-to-speech, and `aria-*` properties are often used to tag elements that have some semantic meaning.  

In [9]:
# You do not need to use regular expressions in this solution. This is only for testing.
import re

def reviews_check(reviews):
    type_check = lambda field, typ: all(field in r and typ(r[field]) for r in reviews)
    test.true(type_check("author", lambda r: isinstance(r, str)))
    test.true(type_check("rating", lambda r: isinstance(r, float)))

    datecheck = re.compile("^\d{4}-\d{2}-\d{2}$")
    test.true(type_check("date", lambda r: datecheck.match(r)))
    test.true(type_check("description", lambda r: isinstance(r, str)))

def parse_page_test(parse_page):
    reviews, num_pages = parse_page(retrieve_html("https://www.yelp.com/biz/the-porch-at-schenley-pittsburgh")[1])
    reviews_check(reviews)
    test.equal(len(reviews), 20)
    test.equal(num_pages, 32)
    
import math
@test
def parse_page(html):

    soup = BeautifulSoup(html, "html.parser")
    
    result = []
    reviews = soup.find_all("script",attrs={"type":"application/ld+json"})[0]
    reviews = json.loads(reviews.contents[0])
    reviews_count = reviews["aggregateRating"]["reviewCount"]
    reviews = reviews["review"]
    for review in reviews:
        tmp = {}
        tmp["author"] = review["author"]
        tmp["rating"] = float(review["reviewRating"]["ratingValue"])
        tmp["date"] = review["datePublished"]
        tmp["description"] = review["description"]
        result.append(tmp)
    
    links = soup.find_all("link",attrs={"rel":"next"},href=True)
    next_link = links[0]["href"] if links else None
    return result, math.ceil(reviews_count/20)

### TESTING parse_page: PASSED 5/6
# 5	: Failed: 33 is not equal to 32
###



---

## Q 3.5: Extract all of the Yelp reviews for a Single Restaurant

So now that we have parsed a single page, and figured out a method to go from one page to the next we are ready to combine these two techniques and actually crawl through web pages! 

Using `requests`, programmatically retrieve __ALL__ of the reviews for a __single__ restaurant (provided as a parameter). Just like the API was paginated, the HTML paginates its reviews (it would be a very long web page to show 300 reviews on a single page) and to get all the reviews you will need to parse and traverse the HTML. As input your function will receive a URL corresponding to a Yelp restaurant. As output return a list of dictionaries (structured the same as question 3 containing the relevant information from the reviews.

Return reviews in the order that they are present on the page.

You will need to get the number of pages on the first request and generate the URL for subsequent pages automatically. Use the Yelp website to see how the URL changes for subsequent pages.

In [9]:
def extract_reviews_test(extract_reviews):
    reviews = extract_reviews("https://www.yelp.com/biz/larry-and-carols-pizza-pittsburgh")
    test.equal(len(reviews), 47) # This may change!
    reviews_check(reviews)

@test
def extract_reviews(url):
    """
    Retrieve ALL of the reviews for a single restaurant on Yelp.

    Parameters:
        url (string): Yelp URL corresponding to the restaurant of interest.

    Returns:
        reviews (list): list of dictionaries containing extracted review information
    """
    tmp, pages = parse_page(retrieve_html(url)[1])
    res = []
    for i in range(pages):
        if i>0:
            current, tmp_count = parse_page(retrieve_html(url+"?start="+str(20*i))[1])
        else:
            current = tmp
        res+=current
    
    return res


### TESTING extract_reviews: PASSED 4/5
# 0	: Failed: 46 is not equal to 47
###

